In [ ]:
# !pip install geopandas metpy xarray[complete] -q
# !pip install eccodes cfgrib rioxarray ecmwf.opendata 

In [ ]:
import xarray as xr
import cfgrib

# import cartopy.crs as ccrs
# import matplotlib.pyplot as plt
# from cartopy.io.shapereader import Reader
# import cartopy.io.shapereader as shpreader

# from cartopy.feature import ShapelyFeature
# import cartopy.feature as cf
import os 
from ecmwf.opendata import Client
# plt.style.available
#import colormaps as cmaps 
import concurrent
import numpy as np

# import cartopy.crs as crs
# import cartopy.feature as cfeature
# import geopandas as gpd


import numpy as np

# Libraries to assist with animation and visualisations
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# Disable warnings for data download via API
import urllib3
urllib3.disable_warnings()
client = Client(source="ecmwf")

import metpy.calc as mpcalc
from metpy.units import units

import json
import pandas as pd
from datetime import datetime, timedelta

import rioxarray

import xarray as xr
import cfgrib
# import eccodes
# import geopandas as gpd
# from shapely.geometry import Point
import json
import zipfile
import os
from os import walk



#file='dist.shp'
#gdf = gpd.read_file(file)
#gdf = gdf.to_crs("EPSG:4326")
#geometry_column = gdf['geometry']

# out="dist.shp"
# gdf.geometry[:].to_file(out)
# gdf.to_file("dist.geojson", driver='GeoJSON')

# od=ShapelyFeature(Reader('IND.shp').geometries(), ccrs.PlateCarree(),
#                                 linewidth = 1,facecolor='none', edgecolor=[0,0,0], alpha=0.7, )

In [2]:
# HRES:

# fc: Forecast.
# ENS:

# cf: Control forecast.
# pf: Perturbed forecast.
# em: Ensemble mean.
# es: Ensemble standard deviation.
# ep: Probabilities.
# Valid values for stream are:

# oper: Atmospheric fields from HRES - 00 UTC and 12 UTC.
# wave: Ocean wave fields from HRES - 00 UTC and 12 UTC.
# enfo: Atmospheric fields from ENS.
# waef: Ocean wave fields from ENS.
# scda: Atmospheric fields from HRES - 06 UTC and 18 UTC.
# scwv: Ocean wave fields from HRES - 06 UTC and 18 UTC.

In [ ]:
step = list(range(3, 91, 3))


client.retrieve( date=0,
    time=12,
    type="fc",
    stream='oper',step=step,
    param=['tp','tcwv',"msl","2t",'10u',"10v"],
    target="atms.grib2",
)




<multiple>:   0%|          | 0.00/127M [00:00<?, ?B/s]

<multiple>:   0%|          | 0.00/170M [00:00<?, ?B/s]

In [3]:
# import cfgrib
t2 = xr.open_dataset('atms.grib2',engine='cfgrib',decode_coords="all",
                     backend_kwargs={ 'filter_by_keys':{ 'cfVarName': 't2m',}})['t2m']
ds=xr.open_dataset('atms.grib2',engine='cfgrib')
# ds1=ds.sel(longitude=slice(60,100),latitude=slice(40,0))
rf = ds.sel(longitude=slice(60,100),latitude=slice(40,0))['tp'][:, :, :]  # Select a specific time slice
p =ds.sel(longitude=slice(60,100),latitude=slice(40,0))['msl'][:, :, :]/100  # Select a specific time slice
tcw=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['tcwv']
u10=xr.open_dataset('atms.grib2',engine='cfgrib',backend_kwargs={ 'filter_by_keys':{ 'cfVarName': 'u10',}}).sel(longitude=slice(60,100),latitude=slice(40,0))['u10']
v10=xr.open_dataset('atms.grib2',engine='cfgrib',backend_kwargs={ 'filter_by_keys':{ 'cfVarName': 'v10',}}).sel(longitude=slice(60,100),latitude=slice(40,0))['v10']

w10=(u10**2+v10**2)**0.5
from metpy.units import units
rf1=rf* units('m').to(units.mm)

Ignoring index file 'atms.grib2.52121.idx' older than GRIB file
Ignoring index file 'atms.grib2.5b7b6.idx' older than GRIB file
skipping variable: paramId==167 shortName='t2m'
Traceback (most recent call last):
  File "e:\anaconda3\envs\wrf_plot\lib\site-packages\cfgrib\dataset.py", line 721, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "e:\anaconda3\envs\wrf_plot\lib\site-packages\cfgrib\dataset.py", line 639, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=np.float64(10.0)) new_value=Variable(dimensions=(), data=np.float64(2.0))
Ignoring index file 'atms.grib2.52121.idx' older than GRIB file
Ignoring index file 'atms.grib2.52121.idx' older than GRIB file


In [4]:

def dict_l(data,fm):
  data_dict=data.sel(longitude=slice(60, 100), latitude=slice(38, 0)).squeeze().to_dict(data='list', encoding=False)
  def convert_types(obj):
    if isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
        return obj.isoformat()  # Convert datetime objects to ISO format
    elif isinstance(obj, timedelta):
        return obj.total_seconds()  # Convert timedelta to total seconds
    elif isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert numpy arrays to lists
    elif isinstance(obj, (np.float32, np.float64)):
        return float(obj)  # Convert numpy floats to native floats
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)  # Convert numpy integers to native ints
    raise TypeError(f"Type {type(obj)} not serializable")

  file=f"{fm}.json"
  # Save the dictionary as a JSON file with the extended convert_types function
  with open(file, 'w') as json_file:
      json.dump(data_dict, json_file, indent=4, default=convert_types)

In [ ]:


# def dict_c(data, fm, gdf):
#     """
#     Clips the xarray DataArray to a specific region, converts it to a dictionary, and saves it as a JSON file.
    
#     Parameters:
#     data (xarray.DataArray): The raster data to be clipped and converted to a dictionary.
#     fm (str): The filename (without extension) to save the JSON file.
#     gdf (geopandas.GeoDataFrame): The GeoDataFrame containing the shapefile for clipping.
    
#     Returns:
#     None
#     """
#     # Clip the data using the given GeoDataFrame (gdf)
#     data = data.rio.write_crs("EPSG:4326", inplace=True)  # Ensure CRS is correct
#     if data.rio.crs != gdf.crs:
#         gdf = gdf.to_crs(data.rio.crs)
    
#     clipped_data = data.rio.clip(gdf.geometry, gdf.crs, drop=True).squeeze()
    
#     # Convert the clipped data to a dictionary
#     data_dict = clipped_data.fillna(0).to_dict(data='list', encoding=False)
    

#     # Function to handle non-serializable types
#     def convert_types(obj):
#         if isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
#             return obj.isoformat()  # Convert datetime objects to ISO format
#         elif isinstance(obj, timedelta):
#             return obj.total_seconds()  # Convert timedelta to total seconds
#         elif isinstance(obj, np.ndarray):
#             return obj.tolist()  # Convert numpy arrays to lists
#         elif isinstance(obj, (np.float32, np.float64)):
#             return float(obj)  # Convert numpy floats to native floats
#         elif isinstance(obj, (np.int32, np.int64)):
#             return int(obj)  # Convert numpy integers to native ints
#         raise TypeError(f"Type {type(obj)} not serializable")
    
#     # Save the dictionary as a JSON file
#     file = f"{fm}_contour.json"
#     with open(file, 'w') as json_file:
#         json.dump(data_dict, json_file, indent=4, default=convert_types)

#     print(f"Data successfully saved as {file}")



In [6]:
dict_l(t2,"2m_temp")
dict_l(rf1,"rf")
dict_l(p,"msl")
dict_l(tcw,"tcwv")
# dict(w10,"w10")
# dict(tcw,"tcwv")


In [ ]:
# dict_c(t2,"2m_temp",gdf)
# dict_c(rf1,"rf",gdf)
# dict_c(p,"msl",gdf)
# dict_c(tcw,"tcwv",gdf)


# cropped=t2.rio.write_crs("EPSG:4326", inplace=True).rio.clip(gdf.geometry, gdf.crs, drop=True)
# dfff=optomise_dataframe(cropped[0])

# fig, ax = plt.subplots(figsize=(10, 6))

# # If you're mapping directly from the grid:
# cropped[16].plot(
#     ax=ax,
#     x='longitude',
#     y='latitude',
#     cmap='viridis',  # Choose a color map
#     add_colorbar=True,
#     robust=True,  # Removes extreme outliers
# )
# gdf.boundary.plot(ax=ax, color="black", linewidth=0.5)
# import plotly.express as px

# df = px.data.gapminder().query("year==2007")
# fig = px.choropleth(df, locations="iso_alpha",
#                     color="lifeExp", # lifeExp is a column of gapminder
#                     hover_name="country", # column to add to hover information
#                     color_continuous_scale=px.colors.sequential.Plasma)
# fig.show()

# xr.open_dataset('atms.grib2',engine='cfgrib',backend_kwargs={ 'filter_by_keys':{ 'cfVarName': 'u10',}}).sel(longitude=slice(60,100),latitude=slice(40,0))['u10']


# # Assuming `ds` is your xarray dataset
# data_dict = ds['tp'].sel(longitude=slice(60, 100), latitude=slice(38, 0)).squeeze().to_dict(data='list', encoding=False)
# # data_dict['tcw'] = ds['tcwv'].sel(longitude=slice(60, 100), latitude=slice(38, 0)).squeeze()
# # Extended function to handle all types of non-serializable data
# def convert_types(obj):
#     if isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
#         return obj.isoformat()  # Convert datetime objects to ISO format
#     elif isinstance(obj, timedelta):
#         return obj.total_seconds()  # Convert timedelta to total seconds
#     elif isinstance(obj, np.ndarray):
#         return obj.tolist()  # Convert numpy arrays to lists
#     elif isinstance(obj, (np.float32, np.float64)):
#         return float(obj)  # Convert numpy floats to native floats
#     elif isinstance(obj, (np.int32, np.int64)):
#         return int(obj)  # Convert numpy integers to native ints
#     raise TypeError(f"Type {type(obj)} not serializable")

# # Save the dictionary as a JSON file with the extended convert_types function
# with open('rf.json', 'w') as json_file:
#     json.dump(data_dict, json_file, indent=4, default=convert_types)



# fig = plt.figure(figsize=(12,12)) # Define the figure and specify size
# ax = plt.subplot(1,1,1, projection=ccrs.PlateCarree()) # Specify plot area & projection
# gl = ax.gridlines(draw_labels=True,linewidth=.6, color='gray', alpha=0.5, linestyle='-.')
# gl.xlabel_style = {"size" : 6}
# gl.top_labels = False
# gl.right_labels = False
# gl.ylabel_style = {"size" : 6}
# va=rf[11,:,:]
# # ax.set_title(f'AOD at 550nm, {str(da.forecast_reference_time[0].values)[:-16]}', fontsize=12) # Set figure title
# ax.coastlines(color='black') # Add coastlines
# im = plt.pcolormesh(va.longitude, va.latitude, va, cmap=cmaps.precip2_17lev	, ) # Plot the data
# # plt.contour(da.longitude, da.latitude, va ) # Plot the data
# im.axes.add_feature(od)

# lab=f'Total precipitation of at least 25 mm (%)'
# cbar = plt.colorbar(im,fraction=0.021,shrink=0.85, pad=0.12,extend='min') # Specify the colourbar
# # cbar.set_label(label='pressure(hPa)', size='large', weight='bold')
# # plt.plot(mm[:8], nn[:8], color='m', linewidth=2, transform=ccrs.PlateCarree())  # Line connecting points
# ax.set_xlim(60,100)
# ax.set_ylim(6,38)

# # plt.title(f"Total precipitation of at least 25 mm (%) ,{va.valid_time.dt.strftime('%d-%B-%Y %H:%M').values}")
# ax.text(1.1, -0.012, '@Subhrajit', transform=ax.transAxes, fontsize=12,
#             verticalalignment='bottom', bbox=dict(facecolor='white', alpha=0.8),weight='bold')
# # time=va.valid_time.dt.strftime('%d_%B_%Y_%H').values
# plt.grid(True)
# # plt.scatter(x=mm[:8], y=nn[:8],
# #             color="m",
# #             s=10,
# #             alpha=1,
# #             transform=ccrs.PlateCarree())

# # for i, (lon, lat) in enumerate(zip(mm[:10], nn[:10]),):
# #     plt.text(lon+1, lat, f'({lon}, {lat})', fontsize=12, ha='right', color='g',transform=ccrs.PlateCarree())
    
# # Show the plot with a frame
# plt.gca().spines['top'].set_visible(True)
# plt.gca().spines['right'].set_visible(True)
# plt.gca().spines['left'].set_visible(True)
# plt.gca().spines['bottom'].set_visible(True)

# plt.tight_layout()




# # cbar.set_label(lab) # Define the colourbar label
# def animate(i):
#     array = rf[i,:,:].values
#     # array1 = p[i,:,:].values
#     im.set_array(array1.flatten())
#     # ax.set_title(f'AOD at 550nm')#, {str(da.forecast_reference_time[i].values)[:-16]}', fontsize=12)
#     ax.set_title(f"{p.valid_time[i].dt.strftime('%d-%B-%Y %H:%M').values}",fontsize=12,weight='bold')


# frames =16
# ani = animation.FuncAnimation(fig, animate, frames, interval=150)
# # ani.save('pressure.gif',fps = 90)
# HTML(ani.to_jshtml())